In [ ]:
# STEP 1: install libs
!pip install diffusers transformers accelerate safetensors pillow torch gradio

In [ ]:
# STEP 2: Imports
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, EulerAncestralDiscreteScheduler
from PIL import Image, ImageEnhance
import torch, os, uuid
import gradio as gr

#  STEP 3: Hugging Face Token (Required for Diffusers)
HUGGING_FACE_TOKEN = ""  # Replace if expired

In [ ]:
# STEP 4: Environment
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
print(f"Using device: {DEVICE.upper()}")

# STEP 5: Model Setup
scheduler = EulerAncestralDiscreteScheduler.from_pretrained(
    "runwayml/stable-diffusion-v1-5", subfolder="scheduler"
)

pipe_txt2img = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    scheduler=scheduler,
    torch_dtype=DTYPE,
    use_auth_token=HUGGING_FACE_TOKEN
).to(DEVICE)
pipe_txt2img.enable_attention_slicing()

pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    scheduler=scheduler,
    torch_dtype=DTYPE,
    use_auth_token=HUGGING_FACE_TOKEN
).to(DEVICE)
pipe_img2img.enable_attention_slicing()

In [ ]:
# STEP 6: Image Generation Function
def generate(prompt):
    neg_prompt = "blurry, bad anatomy,blurry, low quality, deformed, malformed, poorly drawn, disfigured, ugly, out of frame low resolution, noisy, out of focus, deformed, distorted, poorly drawn, badly rendered, pixelated, jagged edges, overexposed, underexposed, low contrast, dull colors, wrong proportions, missing limbs, extra limbs, floating objects, inconsistent lighting, unrealistic shadows, poorly lit, oversaturated, flat, grainy, muddy textures, cartoonish, amateur style, unrealistic anatomy, low-poly, depth errors, incorrect perspective, stretched textures, artifacts, watermark, text, logo, cropped, cut-off, out of frame, duplicated, glitch, flickering, poor composition, unrealistic reflections, uncanny faces, plastic-looking, lazy design, placeholder render, low quality, blurry, noisy, out of focus, deformed, distorted, malformed, poorly drawn, badly rendered, pixelated, jagged edges, overexposed, underexposed, low contrast, dull colors, wrong proportions, missing limbs, extra limbs, floating objects, inconsistent lighting, unrealistic shadows, poorly lit, oversaturated, flat, grainy, muddy textures, cartoonish, amateur style, unrealistic anatomy, low-poly, depth errors, incorrect perspective, stretched textures, artifacts, watermark, text, logo, cropped, cut-off, out of frame, duplicated, glitch, flickering, poor composition, unrealistic reflections, uncanny faces, plastic-looking, lazy design, placeholder render, bad hands, bad eyes, noisy background, rushed details, AI artifacts, unrealistic depth, visual compression, poor mesh structure, disfigured body, misshapen geometry, posterized, no coherence, noisy lines, messy hair, poor lighting model, fake realism, soft focus, low resolution, low quality, fuzzy details, smudged textures, distorted edges, washed out, out of focus, pixelated, compression artifacts, poor detail, low fidelity, simplified geometry, undefined edges, over-smoothed, plastic texture, unrealistic skin, low sharpness, soft edges, overexposed, underexposed, grainy, mushy, bad texture mapping, low poly, inaccurate lighting, artificial look, lazy rendering, depthless, fake realism, waxy look, painterly blur, fake shading, noisy output, banding, blotchy surface, hazy background, unrefined shapes, bad anatomy, malformed geometry, wrong lighting model, lack of artistic vision, poor conceptualization, uninspired composition, generic design, unoriginal elements,flat textures, unconvincing materials, artificial sheen, monotonous surfaces, rough surfacing,imprecise rendering, poor object definition, unsharp edges, fuzzy details, blobby objects, simplified forms,unnatural highlights, artificial reflections, unrealistic specularity, over-simplistic lighting, lack of subtle shadows,disjointed elements, staged appearance, lack of authenticity, imperfect transitions, simulated look, visual noise,aliasing, clipping errors, polygon edges, z-fighting, texture stretching, mesh tears, uv map issues, tiling artifacts, unnatural light falloff, over-bloom, light bleeding, inconsistent volumetric effects, poor subsurface scattering, flat ambient light, plastic skin, rubbery textures, unnatural reflections, uniform roughness, sterile surfaces, overly clean, awkward framing, disproportionate elements, lack of depth of field, stiff posing, motion blur artifacts, visual noise, artificial look, computer generated imagery (CGI) look, render farm quality, over-sharpened, gimmicky effects"

    # Text to Image
    with torch.autocast(DEVICE) if DEVICE == "cuda" else torch.no_grad():
        base = pipe_txt2img(prompt=prompt, negative_prompt=neg_prompt,
                            guidance_scale=13, num_inference_steps=60,
                            height=768, width=768).images[0]

    # Img2Img Refinement
    init_image = base.resize((768, 768)).convert("RGB")
    with torch.autocast(DEVICE) if DEVICE == "cuda" else torch.no_grad():
        refined = pipe_img2img(
            prompt=prompt, negative_prompt=neg_prompt,
            image=init_image, strength=0.65,
            guidance_scale=15, num_inference_steps=80
        ).images[0]

    # Enhancement
    refined = ImageEnhance.Contrast(refined).enhance(1.4)
    refined = ImageEnhance.Sharpness(refined).enhance(2.2)
    refined = ImageEnhance.Brightness(refined).enhance(1.1)

    # Upscale
    final = refined.resize((2048, 2048), resample=Image.LANCZOS)

    # Save Image
    os.makedirs("outputs", exist_ok=True)
    filename = f"outputs/image_{uuid.uuid4().hex[:8]}.png"
    final.save(filename)

    return final, filename

In [ ]:
# STEP 7: Gradio UI for Multiple Users
with gr.Blocks() as demo:
    gr.Markdown("## AI Image Generator - Ultra HD Realistic")
    gr.Markdown("Enter your description and get an AI-generated image!")

    with gr.Row():
        prompt = gr.Textbox(label="Enter Prompt", lines=2, placeholder="e.g., A magical forest at sunset with glowing animals")
        generate_btn = gr.Button("Generate Image")

    image_output = gr.Image(label="Final Image", type="pil", interactive=False)
    download_file = gr.File(label="Download Image", interactive=False)

    generate_btn.click(fn=generate, inputs=prompt, outputs=[image_output, download_file])

# STEP 8: Launch Public Gradio App
demo.launch(share=True, show_error=True, debug=True)